### <font color="#003865"> ANOP 203 <br> Introduction to Programming for Business Analytics <br><br>

# <center> <font color="#E87722"> Lectures 14, Part 2: Incremental Formulations and the TSP<br> <img src="https://imgs.xkcd.com/comics/travelling_salesman_problem.png" width="500">

### <center> Thiago Serra, Ph.D. <br><br> Bucknell University <br> Fall 2022

# <font color="#E87722"> Recap from ANOP 203
    
    
## <font color="#003865"> Loading a CSV file using pandas
    
Now we will load a file with the 1,000 largest cities by population in the US, which was obtained from the [ODS website](https://public.opendatasoft.com/explore/dataset/1000-largest-us-cities-by-population-with-geographic-coordinates/table/?sort=-rank). You shoul download an extra file from Moodle to continue.

In [1]:
import pandas as pd

data = pd.read_csv("File for Lecture 14 - 1000-largest-us-cities-by-population-with-geographic-coordinates.csv", sep=";") 
data.head()

,City,Rank,State,Growth From 2000 to 2013,Population,Coordinates
0,Marysville,552,Washington,115.7,63269,"48.0517637,-122.1770818"
1,Perris,466,California,98.7,72326,"33.7825194,-117.2286478"
2,Cleveland,48,Ohio,-18.1,390113,"41.49932,-81.6943605"
3,Worcester,129,Massachusetts,5.8,182544,"42.2625932,-71.8022934"
4,Columbia,192,South Carolina,11.7,133358,"34.0007104,-81.0348144"


## <font color="#003865"> Using dictionaries to simplify things
    
The following dictionary will allow us to shorten the description of a city name and state, which is necessary given that some city names exist across multiple states.

In [2]:
# https://gist.github.com/rogerallen/1583593
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

## <font color="#003865"> Storing tuples in a dictionary

We saw above that we can use tuples as keys for a dictionary. There are cases in which tuples will be the values. For example, if we want to remember the (x,y) coordinates of each city.

In [3]:
city_by_rank = {}
city_coordinates = {}

nb_of_cities = len(data.index)
for i in range(nb_of_cities):
    city_name = data.values[i][0] + "-" + us_state_to_abbrev[data.values[i][2]]
    rank = data.values[i][1]
    coordinate_list = data.values[i][5].split(",")
    x = float(coordinate_list[0])
    y = float(coordinate_list[1])
    
    city_by_rank[rank] = city_name
    city_coordinates[city_name] = (x,y)
    
print(city_by_rank[10])
print(city_coordinates["San Jose-CA"])

San Jose-CA
(37.3382082, -121.8863286)


  ## <font color="#003865"> Calculating distances between (x,y) coordinates

Sometimes a good approximation is all you need. In the code below, I am assuming that the Earth is flat, and I am calculating the distance in the plane between the coordinates and then multiplying it by an adjustment factor to make it match what Google gives for the case between New York and Los Angeles. It turns out that the distance between New York and Pittsburgh stays close yet!

In [4]:
import math

def distance_between_cities(A,B):
    (latA,lonA) = city_coordinates[A]
    (latB,lonB) = city_coordinates[B]
    
    # Adjusted to match New York to Los Angeles; still good approximation for New York to Pittsburgh!
    return 62.36*math.sqrt((latA-latB)*(latA-latB) + (lonA-lonB)*(lonA-lonB))

print(distance_between_cities("New York-NY", "Los Angeles-CA")) # Google: 2789.8
print(distance_between_cities("New York-NY", "Pittsburgh-PA")) # Google: 369.6

2789.7586933994066
373.9183433115823


# <font color="#E87722">Input for Our Optimization Problem
    
If we want to consider the $N$ most populous cities in the US and the shortest tour involving all of them, here is the data for the problem that we want to solve. By choosing a value for $N$ between 1 and 1,000, we produce the instance as described below.

In [5]:
N = 10
cities = [ city_by_rank[i] for i in range(1,N+1) ]
pairs_of_cities = [ (i,j) for i in cities for j in cities if i != j ]
distance = { (city1, city2):distance_between_cities(city1,city2) for (city1, city2) in pairs_of_cities }

# This function is going to help us change the size of the problem we are solving later
def change_problem_size(n):
    global N
    global cities
    global pairs_of_cities
    global distance
    
    N = n
    cities = [ city_by_rank[i] for i in range(1,N+1) ]
    pairs_of_cities = [ (i,j) for i in cities for j in cities if i != j ]
    distance = { (city1, city2):distance_between_cities(city1,city2) for (city1, city2) in pairs_of_cities }

## <font color="#003865"> Remembering solution times
    
This is a dictionary that will be populated as we run different algorithms.

In [6]:
runtime = {}

# <font color="#E87722">Solving the TSP by Coding the Algorithm

## <font color="#003865"> A Recursive Algorithm
    
The cell below shows a recursive method to solve the TSP to optimality, which of course cannot be scale up by much.

In [7]:
import time

# If you are reading the code for the first time, skip this function!
# If not, welcome back: here is where we actually solve the problem
# We start with last_city="New York" and cities_left containing all other cities
# Then we test adding every city next, and solving the resulting subproblem
def shortest_tsp_tour(last_city):
    # If there are no cities left, we connect the last city with the first
    if len(cities_left) == 0:
        # We return a tuple with distance and a list of the last two cities
        return (distance_between_cities(last_city, first_city), [first_city, last_city])
    
    # If there are cities left, we test each one being the next
    shortest_route_value = None
    
    # We cannot loop directly over cities_left if we are changing the set
    for city in set_of_cities:
        # So we check if the city is there before proceeding
        if not city in cities_left:
            continue
            
        # When trying a city, we remove it from the cities left    
        cities_left.remove(city)
        # And we solve the subproblem on the remaining cities
        (subroute_value, subroute_list) = shortest_tsp_tour(city)
        
        # We add the distance from last city to current city
        route_value = subroute_value + distance_between_cities(last_city,city)
        # And we check if this is the first or shortest route so far
        if shortest_route_value is None or route_value < shortest_route_value:
            # If so, we update the shortest route value and route list
            shortest_route_value = route_value
            shortest_route_list = subroute_list.copy()
            shortest_route_list.append(last_city)
        
        # Last, we add the city back to the set before trying the next one
        cities_left.add(city)
    
    # After testing all possibilities, we return the best one
    return (shortest_route_value, shortest_route_list)

# CHANGE NUMBER OF CITIES HERE!
change_problem_size(10)

# It does not matter where we start since we go everywhere
# But since the first city is always in the list, we start from there
first_city = cities[0]

# We create a set with the top ranked cities
set_of_cities = set(cities)

# We make a copy of that set and remove the first city from it
cities_left = set_of_cities.copy()
cities_left.remove(first_city)

# We time the code, and get the optimal value and sequence of cities
time_before = time.time()
(tour_value, tour_list) = shortest_tsp_tour(first_city)
time_after = time.time()

# We print a summary of the run
print("Solving a problem of", N, "to optimality")
print("Best tour value:",tour_value)
print("Best tour list:")
for city in tour_list:
    print("\t",city)
print("Time to solve:", time_after-time_before)

runtime[("Recursive",N)] = time_after-time_before

Solving a problem of 10 to optimality
Best tour value: 6536.262647475204
Best tour list:
	 New York-NY
	 Chicago-IL
	 San Jose-CA
	 Los Angeles-CA
	 San Diego-CA
	 Phoenix-AZ
	 San Antonio-TX
	 Houston-TX
	 Dallas-TX
	 Philadelphia-PA
	 New York-NY
Time to solve: 1.0248908996582031


## <font color="#003865"> Remembering Solved Subproblems
    
The next cell differs from the previous one in only four lines, each of which followed by <b>#NEW!</b> if you want to search for them. These lines help us keep track of subproblems that we have already seen to avoid solving them again. Because the subproblem depends on what was the last city and which cities are left, and the latter is a set, we need to use an immutable version of this set (a <i>frozenset</i>) as our key. 

In [8]:
import time

saved_subproblems = {} #NEW!

# If you are reading the code for the first time, skip this function!
# If not, welcome back: here is where we actually solve the problem
# We start with last_city="New York" and cities_left containing all other cities
# Then we test adding every city next, and solving the resulting subproblem
def shortest_tsp_tour(last_city):
    # If there are no cities left, we connect the last city with the first
    if len(cities_left) == 0:
        # We return a tuple with distance and a list of the last two cities
        return (distance_between_cities(last_city, first_city), [first_city, last_city])
    
    if (last_city,frozenset(cities_left)) in saved_subproblems:      #NEW!
        return saved_subproblems[(last_city,frozenset(cities_left))] #NEW!
    
    # If there are cities left, we test each one being the next
    shortest_route_value = None
    
    # We cannot loop directly over cities_left if we are changing the set
    for city in set_of_cities:
        # So we check if the city is there before proceeding
        if not city in cities_left:
            continue
            
        # When trying a city, we remove it from the cities left    
        cities_left.remove(city)
        # And we solve the subproblem on the remaining cities
        (subroute_value, subroute_list) = shortest_tsp_tour(city)
        
        # We add the distance from last city to current city
        route_value = subroute_value + distance_between_cities(last_city,city)
        # And we check if this is the first or shortest route so far
        if shortest_route_value is None or route_value < shortest_route_value:
            # If so, we update the shortest route value and route list
            shortest_route_value = route_value
            shortest_route_list = subroute_list.copy()
            shortest_route_list.append(last_city)
        
        # Last, we add the city back to the set before trying the next one
        cities_left.add(city)
        
    saved_subproblems[(last_city,frozenset(cities_left))] = (shortest_route_value, shortest_route_list) #NEW!
    
    # After testing all possibilities, we return the best one
    return (shortest_route_value, shortest_route_list)

# CHANGE NUMBER OF CITIES HERE!
change_problem_size(10)

# It does not matter where we start since we go everywhere
# But since the first city is always in the list, we start from there
first_city = cities[0]

# We create a set with the top ranked cities
set_of_cities = set(cities)

# We make a copy of that set and remove the first city from it
cities_left = set_of_cities.copy()
cities_left.remove(first_city)

# We time the code, and get the optimal value and sequence of cities
time_before = time.time()
(tour_value, tour_list) = shortest_tsp_tour(first_city)
time_after = time.time()

# We print a summary of the run
print("Solving a problem of", N, "to optimality")
print("Best tour value:",tour_value)
print("Best tour list:")
for city in tour_list:
    print("\t",city)
print("Time to solve:", time_after-time_before)

runtime[("DP",N)] = time_after-time_before

Solving a problem of 10 to optimality
Best tour value: 6536.262647475204
Best tour list:
	 New York-NY
	 Chicago-IL
	 San Jose-CA
	 Los Angeles-CA
	 San Diego-CA
	 Phoenix-AZ
	 San Antonio-TX
	 Houston-TX
	 Dallas-TX
	 Philadelphia-PA
	 New York-NY
Time to solve: 0.013149738311767578


# <font color="#E87722">Solving the TSP with a MIP Solver

In what follows, we will start solving our problem and adding more constraints to it as needed.

## <font color="#003865"> Figuring out where to go next
    
The model belows decides which other city to visit after each city while minimizing the total cost. The solution does not provide us a route yet. Why? Would the value of the solution represent a lower or upper bound?

In [9]:
import gurobipy as gb

# CHANGE NUMBER OF CITIES HERE!
change_problem_size(10)

model = gb.Model()

connection = model.addVars(pairs_of_cities, obj = distance, vtype = gb.GRB.BINARY)

for city in cities:
    model.addConstr(gb.quicksum(connection[city1,city2] for (city1,city2) in pairs_of_cities if city1==city) == 1)

model.optimize()

for (c1,c2) in pairs_of_cities:
    if connection[c1,c2].X == 1:
        print(c1,'---------->',c2)

Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10 rows, 90 columns and 90 nonzeros
Model fingerprint: 0x2b41ba64
Variable types: 0 continuous, 90 integer (90 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 19349.185455
Presolve removed 10 rows and 90 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 2400.29 19349.2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.400288527336e+03, best bound 2.400288527336e+03, gap 0.0000%
New York-NY ----------> Philadelphia-PA
Los Angeles-CA ---

## <font color="#003865"> Modeling a problem of what city comes next: Assignment
    
In this first formulation, we are deciding where to go from each city. That may lead to subsequent issues, which we will take care of later.

In other words, for a collection of cities indexed by $i = 0$ to $N-1$ with the distance between each pair corresponding to the pair of indices $(i,j), i \neq j$ given by $c_{i j}$, we use a binary decision variable $x_{i j} \in \{0, 1\}$ to determine whether we go from $i$ to $j$. The formulation is as follows:


$\min \sum\limits_{i=0}^{N-1} \sum\limits_{j=0}^{N-1} c_{i j} x_{i j}$

$\sum\limits_{i = 0}^{N-1} x_{i j} = 1 ~ \forall j \in \{0, \ldots, N-1\}$

$\sum\limits_{j = 0}^{N-1} x_{i j} = 1 ~ \forall i \in \{0, \ldots, N-1\}$

$x_{i j} \in \{0, 1\} ~ \forall i \in \{0, \ldots, N-1\}, j \in \{0, \ldots, N-1 \} \setminus \{ i \}$

## <font color="#003865"> Adding one more constraint
    
If you solve the initial model for a small size, say $N=5$, you will notice that the the second set of assignment constraints is missing because we get a solution that does not satisfy it.

In the cell below, you should extend the model implementation to include those constraints.

In [10]:
import gurobipy as gb

# CHANGE NUMBER OF CITIES HERE!
change_problem_size(10)

model = gb.Model()

connection = model.addVars(pairs_of_cities, obj = distance, vtype = gb.GRB.BINARY)

for city in cities:
    model.addConstr(gb.quicksum(connection[city1,city2] for (city1,city2) in pairs_of_cities if city1==city) == 1)
    model.addConstr(gb.quicksum(connection[city1,city2] for (city1,city2) in pairs_of_cities if city2==city) == 1)

model.optimize()

for (c1,c2) in pairs_of_cities:
    if connection[c1,c2].X == 1:
        print(c1,'---------->',c2)
        

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 20 rows, 90 columns and 180 nonzeros
Model fingerprint: 0x7053c239
Variable types: 0 continuous, 90 integer (90 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 13115.273601
Presolve time: 0.00s
Presolved: 20 rows, 90 columns, 180 nonzeros
Variable types: 0 continuous, 90 integer (90 binary)

Root relaxation: objective 3.428732e+03, 17 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3428.7316214 3428.73162  0.00%     -    0s

Explored 1 nodes (17 simplex iterations) in 0.01 seconds (0.00 work units

## <font color="#003865"> Printing the visited cities in order
    
The cell below produces the tour in the solution starting from New York. If we do not visit all the cities before going back to New York, we know that something else is missing yet from the formulation.

In [11]:
def print_tour_from_NY():
    starting_point = "New York-NY"
    current_location = starting_point
    while True:
        print(current_location)
        for city in cities:
            if city != current_location and connection[current_location, city].X > 0.9:
                current_location = city
                break
        if current_location == starting_point:
            break
            
print_tour_from_NY()

New York-NY
Philadelphia-PA


## <font color="#003865"> Adding more constraints to avoid infeasible solutions
    
What specific constraint would prevent the infeasibility found above? Extend the model to include such a constraint. For example, by avoiding to go back-and-forth between pairs of cities?

In [12]:
import gurobipy as gb

# CHANGE NUMBER OF CITIES HERE!
change_problem_size(10)

model = gb.Model()

connection = model.addVars(pairs_of_cities, obj = distance, vtype = gb.GRB.BINARY)

for city in cities:
    model.addConstr(gb.quicksum(connection[city1,city2] for (city1,city2) in pairs_of_cities if city1==city) == 1)
    model.addConstr(gb.quicksum(connection[city1,city2] for (city1,city2) in pairs_of_cities if city2==city) == 1)

model.addConstr(connection[("New York-NY", "Philadelphia-PA")]
                +connection[("Philadelphia-PA", "New York-NY")]
                <= 1
)

model.addConstr(connection[("Houston-TX", "Los Angeles-CA")]
                +connection[("Los Angeles-CA", "Houston-TX")]
                <= 1
)
for (city1, city2) in pairs_of_cities:
    if city1 < city2:
        model.addConstr(
            connection[(city1, city2)]
            +connection[(city2, city1)]
            <= 1
        )

model.optimize()

for (c1,c2) in pairs_of_cities:
    if connection[c1,c2].X == 1:
        print(c1,'---------->',c2)
        

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 67 rows, 90 columns and 274 nonzeros
Model fingerprint: 0xc3e7d16c
Variable types: 0 continuous, 90 integer (90 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 11380.955648
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 65 rows, 90 columns, 270 nonzeros
Variable types: 0 continuous, 90 integer (90 binary)

Root relaxation: objective 3.755874e+03, 22 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3755.8740428 3755.87404  0.00%     -    0s

Explored 1 nodes (22 simplex iterat

What constraint would you add next? Extend the model below. For example, how can you avoid a specific cycle consisting of three cities?

# <font color="#E87722">Subtour Elimination Constraints

In what follows, our purpose is to add a set of constraints commonly known as the subtour elimination constraints. The set of all cities is given by $N := \{1, 2, \ldots, n\}$, $S$ is a given subset of those cities, and $\bar{S} := N \setminus S$ corresponds to the set of all cities not in $S$. For every set $S$ which is not empty and not the same as $N$, there should be at least one arc from a city in $S$ to a city in $\bar{S}$ (and vice-versa).

## <font color="#003865">Our first subtour elimination constraint

Can you change the model above to use a single constraint to avoid the trio of cities that insists in showing up? 

## <font color="#003865">Looping over subsets

The piece of code below loops over every subset of a given list. How can you change it to skip the empty set (first one) and the set proper (last one)?

In [13]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

for x in powerset([1,2,3,4]):
    print(x)

()
(1,)
(2,)
(3,)
(4,)
(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)
(1, 2, 3)
(1, 2, 4)
(1, 3, 4)
(2, 3, 4)
(1, 2, 3, 4)


Use the code above to list all proper subsets of the cities in the problem.

## <font color="#003865">Adding the subtour elimination constraints
   
With the help of the method above, extend the formulation to include every possible subtour elimination constraint. Then answer the questions in the following Markdown cell.

## <font color="#003865">Questions:</font>
- How far can you increase the size of the problems that you can solve?
- What is the bottleneck making it considerably slower as the number of cities increase?
- For every set $S$ with lots of cities, there is a complementary set $\bar{S}$ with very few cities. How can you leverage that to reduce the number of subtour elimination constraints?

# <font color="#E87722">Lazy Constraints and Callbacks
    
Sometimes the best way to cope with too many constraints is to simply ignore them until they are needed.

## <font color="#003865">Removing feasible solutions just because</font>

The example below shows an example in which each of the first 10 supposedly feasible solutions is immediately removed through the use of a lazy constraint through the callback function. Only the initial constraint of the model is included.

In [14]:
n = 10
cities = [ city_by_rank[i] for i in range(1,n+1) ]
pairs_of_cities = [ (city1, city2) for city1 in cities for city2 in cities if city1 != city2 ]
distance = { (city1, city2):distance_between_cities(city1,city2) for (city1, city2) in pairs_of_cities }

solutions_removed = 0
def callback_function(model, where):
    global solutions_removed
    if where == gb.GRB.Callback.MIPSOL:
        connection_v = model.cbGetSolution(connection)
        assigned_pairs = []
        for (c1,c2) in pairs_of_cities:
            if connection_v[c1, c2] >= 0.9:
                assigned_pairs.append((c1,c2))
        if solutions_removed < 10:
            print("Removing", assigned_pairs)
            model.cbLazy(gb.quicksum(connection[c1,c2] for (c1,c2) in assigned_pairs) <= len(assigned_pairs)-1)
            solutions_removed += 1


import gurobipy as gb

model = gb.Model()

model.params.LazyConstraints = 1

connection = model.addVars(pairs_of_cities, obj = distance, vtype = gb.GRB.BINARY)

for city in cities:
    model.addConstr(gb.quicksum(connection[city1,city2] for (city1,city2) in pairs_of_cities if city1==city) == 1)

model.optimize(callback_function)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10 rows, 90 columns and 90 nonzeros
Model fingerprint: 0x2b41ba64
Variable types: 0 continuous, 90 integer (90 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Removing [('New York-NY', 'Los Angeles-CA'), ('Los Angeles-CA', 'New York-NY'), ('Chicago-IL', 'New York-NY'), ('Houston-TX', 'New York-NY'), ('Philadelphia-PA', 'New York-NY'), ('Phoenix-AZ', 'New York-NY'), ('San Antonio-TX', 'New York-NY'), ('San Diego-CA', 'New York-NY'), ('Dallas-TX', 'New York-NY'), ('San Jose-CA', 'New York-NY')]
Presolve time: 0.00s
Presolved: 10 rows, 90 columns, 90 nonzeros
Variable types: 0 continuous, 90 integer (90 binary)
Removing [('New York-NY', 'San Jose-CA'), ('Los Angeles-CA',

## <font color="#003865">Adding lazy subtour elimination constraints</font>

Modify the callback to do something useful: for every solution obtained which is not feasible, add the corresponding subtour elimination constraint. You should already have the other two constraints in place.

## <font color="#003865">Questions:</font>
- How far can you increase the size of the problems that you can solve?
- What is the bottleneck making it considerably slower as the number of cities increase?

## <font color="#003865">Comparing runtimes</font>

Go back and modify your implementations to measure runtimes of the model with all subtour elimination constraints, as well as the one that uses a callback to add them as needed. Then compare the runtime with those models with the runtime using the other algorithms.